In [0]:
#Importing Modules:
import csv
import numpy as np
import pandas as pd

from numpy.random import RandomState
from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from prettytable import PrettyTable

In [19]:
#Reading Data using Pandas dataframe and printing 8 rows
#Making last column header as label.
df = pd.read_csv('spambase.data', header=None)
df.rename(columns={57:'label'}, inplace=True)
df[:8]
df.shape


(4601, 58)

In [20]:
# Dividing data into spam and not spam and then into Train/Test:

spam = df[df['label'] == 1]
notspam = df[df['label'] == 0]
spam[:2]
notspam[:3]

spam_train, spam_test = train_test_split(spam, train_size=0.75)
notspam_train, notspam_test = train_test_split(notspam, train_size=0.75)



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [21]:
# Appending data to have train consist of both Spam and not spam:::


X_train = notspam_train.append(spam_train)
X_train.sample(frac=1)
Y_train = X_train.pop('label')
X_test = notspam_test.append(spam_test)
X_test.sample(frac=1)
Y_test = X_test.pop('label')
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)



(3450, 57)
(1151, 57)
(3450,)
(1151,)


In [22]:
# With help from Mult Baysian
spam_bayes_mult = MultinomialNB()
spam_bayes_mult.fit(X_train, Y_train)
sc_train = spam_bayes_mult.score(X_train, Y_train)
sc_test = spam_bayes_mult.score(X_test, Y_test)
print("Train Score is: ", sc_train)
print("Test Score is: ", sc_test)

Train Score is:  0.7979710144927536
Test Score is:  0.7862728062554301


In [23]:
#With help from Bernoulli distribution
spam_bayes_bern = BernoulliNB()
spam_bayes_bern.fit(X_train, Y_train)
sc_train_bern = spam_bayes_bern.score(X_train, Y_train)
sc_test_bern = spam_bayes_bern.score(X_test, Y_test)
print("Train Score is: ", sc_train_bern)
print("Test Score is: ", sc_test_bern)

Train Score is:  0.884927536231884
Test Score is:  0.8922675933970461


In [24]:
#Trying K-fold Validation. Total Examples = 4601
#Using K = 20. 
df.sample(frac=1)
print(df.shape)
data_y = df.pop('label')
data = df.values
print("After Pop:")
y = data_y.values
print("Converted to Numpy, Printing Shapes")
print(np.shape(data))     #Shape has dropped label, this is X (4601, 57)
print(np.shape(y))   # These are labels (4601,)
kf = KFold(n_splits=20, random_state=None, shuffle=False)



(4601, 58)
After Pop: 
Converted to Numpy, Printing Shapes
(4601, 57)
(4601,)


In [46]:
# Function to calulate the rates
def perf_measure(y_actual, y_hat):
    FP = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1

        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(FP/len(y_hat), FN/len(y_hat), (FP+FN)/len(y_hat))
  
#
predicted_y,expected_y = [],[]
k = 0
falsePositives_total = 0.0
falseNegatives_total = 0.0
errors_total = 0
table = PrettyTable(['Fold', 'False Positive Rate', 'False Negative Rate', 'Error Rate'])

for train_indices, test_indices in kf.split(data):
  train_x = (data[train_indices])
  train_y = (data_y[train_indices])
  
  #Fit, Bernoulli distribution has better accuracy
  spam_bayes_bern.fit(train_x, train_y)
  
  #Predicted and Expected Resutls for the current fold
  y_actual = data_y[test_indices].values
  y_hat = spam_bayes_bern.predict(data[test_indices])
  
  #False Positives and False Negatives
  results = perf_measure(y_actual, y_hat)
  
  #Adding rows to results table
  table.add_row([k+1, results[0], results[1], results[2]])
  
  #store results for calulating average rates
  falsePositives_total = falsePostives_total + results[0]
  falseNegatives_total = falseNegatives_total + results[1]
  errors_total = errors_total + results[2]
  k = k + 1
  
  # store result from classification
  predicted_y.extend(y_hat)

  # store expected result
  expected_y.extend(y_actual)
  
  
table.add_row(['Average', falsePositives_total/20, falseNegatives_total/20, errors_total/20])
print(t)



+---------+----------------------+---------------------+----------------------+
|   Fold  | False Positive Rate  | False Negative Rate |      Error Rate      |
+---------+----------------------+---------------------+----------------------+
|    1    |         0.0          | 0.19913419913419914 | 0.19913419913419914  |
|    2    |         0.0          |  0.1608695652173913 |  0.1608695652173913  |
|    3    |         0.0          | 0.20869565217391303 | 0.20869565217391303  |
|    4    |         0.0          | 0.16521739130434782 | 0.16521739130434782  |
|    5    |         0.0          | 0.11304347826086956 | 0.11304347826086956  |
|    6    |         0.0          | 0.16956521739130434 | 0.16956521739130434  |
|    7    |         0.0          | 0.23043478260869565 | 0.23043478260869565  |
|    8    |         0.0          | 0.24347826086956523 | 0.24347826086956523  |
|    9    | 0.04782608695652174  |         0.0         | 0.04782608695652174  |
|    10   | 0.02608695652173913  |      